In [1]:
import os

In [2]:
%pwd

'd:\\Data_science\\Projects\\chicken_Disease_classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Data_science\\Projects\\chicken_Disease_classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen= True)
class PrepareCallbacksConfig:
    root_dir:Path
    tensorboard_root_log_dir:Path
    checkpoint_model_filepath:Path

In [6]:
from CNNCDC.constants import*
from CNNCDC.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        """
        Initializes the ConfigurationManager by loading configuration and
        parameters from YAML files.
        """
        # Load the main configuration file
        self.config = read_yaml(config_filepath)
        # Load the model hyperparameters
        self.params = read_yaml(params_filepath)

        # Create the root artifacts directory if it doesn't exist
        create_directories([self.config.artifacts_root])

    def get_prepare_callbacks_config(self)-> PrepareCallbacksConfig:
        config= self.config.prepare_callbacks
        model_ckpt_dir= os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callbacks_config



In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [9]:
class PrepareCallbacks:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _creat_tb_callbacks(self):
        timestamp= time.strftime("%Y-%m-%d-%H-%S")
        tb_runing_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir, 
            f"tb_logs_at_(timestamp)"
        )

        return tf.keras.callbacks.TensorBoard(log_dir=tb_runing_log_dir)
    
    @property
    def _creat_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath= self.config.checkpoint_model_filepath,
            save_best_only=True
        )
    

    def get_tb_ckpt_callbacks(self):
        return[
            self._creat_tb_callbacks,
            self._creat_ckpt_callbacks
        ]

In [10]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallbacks(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

except Exception as e :
    raise e

[2025-09-26 01:31:31,829: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-26 01:31:31,831: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-26 01:31:31,833: INFO: common: created directory at: artifacts]
[2025-09-26 01:31:31,835: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2025-09-26 01:31:31,837: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
